In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from eofs.xarray import Eof

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib import colors

In [2]:
# Utilities for normalizing the emissions data
min_co2 = 0.
max_co2 = 2400
min_ch4 = 0.
max_ch4 = 0.6
def normalize_co2(data): return data / max_co2
def un_normalize_co2(data): return data * max_co2
def normalize_ch4(data): return data / max_ch4
def un_normalize_ch4(data): return data * max_ch4

def get_rmse(truth, pred):
    weights = np.cos(np.deg2rad(truth.lat))
    return np.sqrt(((truth-pred)**2).weighted(weights).mean(['lat', 'lon', 'sample', 'time'])).data

###### set path for data here:
data_path = "./dataset/"

In [3]:
# Combine historical + ssp585
X = xr.open_mfdataset([data_path + 'inputs_historical.nc', data_path + 'inputs_ssp585.nc']).compute()
# Take the 2nd ensemble member for the historical (the first one has some missing DTR values for some reason...) and the 1st one for SSP585
Y = xr.concat([xr.open_dataset(data_path + 'outputs_historical.nc').sel(member=2), xr.open_dataset(data_path + 'outputs_ssp585.nc').sel(member=1)], dim='time').compute()

# Convert the precip values to mm/day
Y["pr"] *= 86400
Y["pr90"] *= 86400

In [4]:
# Get test data (SSP 245)
test_Y = xr.open_dataset(data_path + 'outputs_ssp245.nc').compute()
test_X = xr.open_dataset(data_path + 'inputs_ssp245.nc').compute()

### Dimensionality Reduction

In [5]:
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
bc_solver = Eof(X['BC'])

# Retrieve the leading EOF, expressed as the correlation between the leading
# PC time series and the input SST anomalies at each grid point, and the
# leading PC time series itself.
bc_eofs = bc_solver.eofsAsCorrelation(neofs=5)
bc_pcs = bc_solver.pcs(npcs=5, pcscaling=1)

In [6]:
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
so2_solver = Eof(X['SO2'])

# Retrieve the leading EOF, expressed as the correlation between the leading
# PC time series and the input SST anomalies at each grid point, and the
# leading PC time series itself.
so2_eofs = so2_solver.eofsAsCorrelation(neofs=5)
so2_pcs = so2_solver.pcs(npcs=5, pcscaling=1)

In [7]:
# Convert the Principle Components of the aerosol emissions (calculated above) in to Pandas DataFrames
bc_df = bc_pcs.to_dataframe().unstack('mode')
bc_df.columns = [f"BC_{i}" for i in range(5)]

so2_df = so2_pcs.to_dataframe().unstack('mode')
so2_df.columns = [f"SO2_{i}" for i in range(5)]

In [8]:
# Bring the emissions data back together again and normalise
leading_historical_inputs = pd.DataFrame({
    "CO2": normalize_co2(X["CO2"].data),
    "CH4": normalize_ch4(X["CH4"].data)
}, index=X["CO2"].coords['time'].data)

# Combine with aerosol EOFs
leading_historical_inputs=pd.concat([leading_historical_inputs, bc_df, so2_df], axis=1)

In [9]:
leading_historical_inputs.head()

,CO2,CH4,BC_0,BC_1,BC_2,BC_3,BC_4,SO2_0,SO2_1,SO2_2,SO2_3,SO2_4
1850,0.000078,0.052177,-0.898978,-0.444571,-2.071612,-0.111809,-0.690692,-0.648146,-0.970250,-0.865609,-0.722246,-1.334943
1851,0.000157,0.052903,-0.897957,-0.445609,-2.073209,-0.143129,-0.629810,-0.650408,-0.965991,-0.863111,-0.733084,-1.328654
1852,0.000239,0.053630,-0.902637,-0.437179,-2.040282,-0.128963,-0.619896,-0.647632,-0.965648,-0.859436,-0.732103,-1.322698
1853,0.000325,0.054356,-0.907249,-0.428673,-2.007596,-0.115736,-0.604771,-0.645783,-0.965185,-0.857202,-0.730953,-1.316006
1854,0.000425,0.055082,-0.936984,-0.381277,-1.826662,0.098731,-0.772527,-0.621943,-0.981160,-0.853043,-0.670587,-1.298459


### Building Model

In [10]:
# import AFTER data read in and processed. Having issues with reading in data
import esem
from esem import gp_model
from esem.data_processors import Whiten, Normalise

# very simple GP with default kernel assuming all years are independant
tas_gp = gp_model(leading_historical_inputs, Y["tas"])
tas_gp.train()

pr_gp = gp_model(leading_historical_inputs, Y["pr"])
pr_gp.train()

dtr_gp = gp_model(leading_historical_inputs, Y["diurnal_temperature_range"])
dtr_gp.train()

pr90_gp = gp_model(leading_historical_inputs, Y["pr90"])
pr90_gp.train()

2025-11-18 23:20:10.999482: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-18 23:20:11.045001: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-18 23:20:12.408063: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


2025-11-18 23:20:14.908229: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [11]:
# build test set
tas_truth = test_Y["tas"].mean('member')
pr_truth = test_Y["pr"].mean('member') * 86400
pr90_truth = test_Y["pr90"].mean('member') * 86400
dtr_truth = test_Y["diurnal_temperature_range"].mean('member')

In [12]:
test_inputs = pd.DataFrame({
    "CO2": normalize_co2(test_X["CO2"].data),
    "CH4": normalize_ch4(test_X["CH4"].data)
}, index=test_X["CO2"].coords['time'].data)

# Combine with aerosol EOFs
test_inputs=pd.concat([test_inputs, 
                       bc_solver.projectField(test_X["BC"], neofs=5, eofscaling=1).to_dataframe().unstack('mode').rename(columns={i:f"BC_{i}" for i in range(5)}),
                       so2_solver.projectField(test_X["SO2"], neofs=5, eofscaling=1).to_dataframe().unstack('mode').rename(columns={i:f"_{i}" for i in range(5)}),
                       ], axis=1)

### Test Predictions

In [13]:
m_tas, _ = tas_gp.predict(test_inputs)
m_pr, _ = pr_gp.predict(test_inputs)
m_pr90, _ = pr90_gp.predict(test_inputs)
m_dtr, _ = dtr_gp.predict(test_inputs)

In [14]:
print(f"RMSE: {get_rmse(tas_truth, m_tas)}")
print(f"RMSE: {get_rmse(dtr_truth, m_dtr)}")
print(f"RMSE: {get_rmse(pr_truth, m_pr)}")
print(f"RMSE: {get_rmse(pr90_truth, m_pr90)}")

RMSE: 0.8028347857163969
RMSE: 0.18046050920739434
RMSE: 0.6111413230340249
RMSE: 1.7519746808710208


### GP kernels
-  model = gp_model(X_train, Y_train, kernel=['Linear', 'Cosine'], kernel_op='add')

- kernel
    - This controls the covariance (or kernel) function of the Gaussian Process.
    - By default, ESEm uses a combination of linear, RBF, and polynomial kernels.
- kernel_op
    - this determines how multiple kernels are combined: 'add' (sum of kernels) or 'mul' (product of kernels).
    - This is helpful if you want more expressive covariance functions (e.g., a periodic part times a linear trend).

chatgpt says these are common 'good combos':
- ['RBF']
- ['Matern32']
- ['Cosine', 'Linear']
- ['RBF', 'Linear']
- ['Matern52', 'Cosine']
Documentation can be found here: https://esem.readthedocs.io/en/latest/emulation.html

In [15]:
def model_kernels(colonel, op):
    """
    Train GP models for different climate variables using specified kernel configuration.
    Returns RMSEs so that we can compare kernels.
    """
    models = {
        "tas": gp_model(leading_historical_inputs, Y["tas"], kernel=colonel, kernel_op=op),
        "pr": gp_model(leading_historical_inputs, Y["pr"], kernel=colonel, kernel_op=op),
        "dtr": gp_model(leading_historical_inputs, Y["diurnal_temperature_range"], kernel=colonel, kernel_op=op),
        "pr90": gp_model(leading_historical_inputs, Y["pr90"], kernel=colonel, kernel_op=op) }

    for m in models.values():
        m.train()
    preds = {name: model.predict(test_inputs)[0] for name, model in models.items()}

    rmse = {
        "tas": get_rmse(tas_truth, preds["tas"]),
        "dtr": get_rmse(dtr_truth, preds["dtr"]),
        "pr": get_rmse(pr_truth, preds["pr"]),
        "pr90": get_rmse(pr90_truth, preds["pr90"]) }

    print(f"RMSE for kernel={colonel}, op={op}: {rmse}")
    return rmse

In [16]:
def testModels():
    kernel_candidates = [['Linear', 'Polynomial'], ['Cosine', 'Linear'], ['RBF'], ['Matern32'], ['RBF', 'Linear'], ['Matern52', 'Cosine']]
    ops = ['add', 'mul']
    results = []

    for k in kernel_candidates:
        for op in ops:
            rmse = model_kernels(k, op)
            results.append((k, op, rmse))
    return

# This can take 60+ minutes fair warning
testModels()

RMSE for kernel=['Linear', 'Polynomial'], op=add: {'tas': array(0.8069459), 'dtr': array(0.18208877), 'pr': array(0.61118874), 'pr90': array(1.75199018)}
RMSE for kernel=['Linear', 'Polynomial'], op=mul: {'tas': array(0.83033842), 'dtr': array(0.18183894), 'pr': array(0.63177948), 'pr90': array(1.81883311)}
RMSE for kernel=['Cosine', 'Linear'], op=add: {'tas': array(0.82998233), 'dtr': array(0.18213001), 'pr': array(0.61112412), 'pr90': array(1.7519892)}
RMSE for kernel=['Cosine', 'Linear'], op=mul: {'tas': array(2.09944838), 'dtr': array(0.2870564), 'pr': array(0.64439374), 'pr90': array(1.86746793)}
RMSE for kernel=['RBF'], op=add: {'tas': array(0.81838417), 'dtr': array(0.17785812), 'pr': array(0.60407864), 'pr90': array(1.73652377)}
RMSE for kernel=['RBF'], op=mul: {'tas': array(0.81838417), 'dtr': array(0.17785812), 'pr': array(0.60407864), 'pr90': array(1.73652377)}
RMSE for kernel=['Matern32'], op=add: {'tas': array(0.80361186), 'dtr': array(0.17758645), 'pr': array(0.61006515),

In [17]:
def plot_gp_comparison(truth, emulated, varname, cmap="coolwarm", diff_cmap="PRGn", savefig=False):

    # Create a figure with three side-by-side subplots
    fig, axes = plt.subplots(1, 3, figsize=(21, 6), subplot_kw={"projection": ccrs.Robinson()})
    divnorm=colors.TwoSlopeNorm(vmin=-2., vcenter=0., vmax=5)

    # --- True ---
    ax = axes[0]
    truth.sel(time=2050).plot(ax=ax, cmap=cmap, norm=divnorm, transform=ccrs.PlateCarree(), cbar_kwargs={"label":"Temperature change / K"})
    ax.set_title(f"{varname} True 2050")
    ax.coastlines()

    # --- Emulated ---
    ax = axes[1]
    emulated.sel(sample=35).plot(ax=ax, cmap=cmap, norm=divnorm, transform=ccrs.PlateCarree(), cbar_kwargs={"label":"Temperature change / K"})
    ax.set_title(f"{varname} Emulated 2050")
    ax.coastlines()

    # --- Difference ---
    ax = axes[2]
    diff = truth.sel(time=2050) - emulated.sel(sample=35)
    diff.plot(ax=ax, cmap=diff_cmap, norm=divnorm, transform=ccrs.PlateCarree(), cbar_kwargs={"label":"Temperature change / K"})
    ax.set_title(f"{varname} Difference (True − Emulated)")
    ax.coastlines()

    # Main title
    fig.suptitle(f"{varname} - Gaussian Process Comparison", fontsize=18)
    fig.tight_layout()

    # Save figure
    if savefig:
        fname = f"gp_{varname}_comparison.png"
        fig.savefig(fname, dpi=250, bbox_inches="tight")
        print(f"Saved figure: {fname}")

    plt.show()
    return

In [ ]:
# generally best kernel is RBF so we use it to visualize
tas_gp = gp_model(leading_historical_inputs, Y["tas"], kernel=['RBF'], kernel_op='add')
tas_gp.train()

m_tas, _ = tas_gp.predict(test_inputs)

# pr_gp = gp_model(leading_historical_inputs, Y["pr"], kernel=['RBF'], kernel_op='add')
# dtr_gp = gp_model(leading_historical_inputs, Y["diurnal_temperature_range"], kernel=['RBF'], kernel_op='add')
# pr90_gp = gp_model(leading_historical_inputs, Y["pr90"], kernel=['RBF'], kernel_op='add')
# pr_gp.train()
# dtr_gp.train()
# pr90_gp.train()

# m_pr, _ = pr_gp.predict(test_inputs)
# m_pr90, _ = pr90_gp.predict(test_inputs)
# m_dtr, _ = dtr_gp.predict(test_inputs)

# hardcoded to show 2050 btw
plot_gp_comparison(tas_truth, m_tas, "tas", cmap="coolwarm", diff_cmap="PRGn", savefig=True)